In [1]:
import os
import sys
from dotenv import load_dotenv
load_dotenv()
import sys
sys.path.insert(1, os.getenv('LIBRARY_PATH'))
from datetime import date
from dateutil.relativedelta import relativedelta
import scrapper
import data_pipeline

In [2]:
url = "https://www150.statcan.gc.ca/t1/tbl1/en/tv.action?pid=1410022301&pickMembers%5B0%5D=2.2&pickMembers%5B1%5D=3.35&cubeTimeFrame.startMonth=06&cubeTimeFrame.startYear=2022&cubeTimeFrame.endMonth=10&cubeTimeFrame.endYear=2022&referencePeriods=20220601%2C20221001"

table = "1410022301"
def clean_string(string):
    return string.replace(" ", "_").replace(",","").replace("(","").replace(")","").replace("-","_").replace("__","_").lower()[:60]

In [3]:

industries_dict = {'3.1': 'industrial_aggregate_including_unclassified_businesses', '3.2': 'industrial_aggregate_excluding_unclassified_businesses', '3.3': 'goods_producing_industries', '3.4': 'forestry_logging_and_support', '3.10': 'mining_quarrying_and_oil_and_gas_extraction', '3.17': 'utilities', '3.21': 'construction', '3.34': 'manufacturing', '3.143': 'service_producing_industries', '3.144': 'trade', '3.215': 'transportation_and_warehousing', '3.253': 'information_and_cultural_industries', '3.267': 'finance_and_insurance', '3.284': 'real_estate_and_rental_and_leasing', '3.295': 'professional_scientific_and_technical_services', '3.306': 'management_of_companies_and_enterprises', '3.321': 'educational_services', '3.331': 'health_care_and_social_assistance', '3.354': 'arts_entertainment_and_recreation', '3.367': 'accommodation_and_food_services', '3.377': 'other_services_except_public_administration', '3.393': 'public_administration'}
pick_members_2 = {"names": ['2.2'], "values": ['average_weekly_earnings']}
pick_members_3 = {"names": ['3.1', '3.2', '3.3', '3.4', '3.10', '3.17', '3.21', '3.34', '3.143', '3.144', '3.215', '3.253', '3.267', '3.284', '3.295', '3.306', '3.321', '3.331', '3.354', '3.367', '3.377', '3.393'],"values": ['industrial_aggregate_including_unclassified_businesses', 'industrial_aggregate_excluding_unclassified_businesses', 'goods_producing_industries', 'forestry_logging_and_support', 'mining_quarrying_and_oil_and_gas_extraction', 'utilities', 'construction', 'manufacturing', 'service_producing_industries', 'trade', 'transportation_and_warehousing', 'information_and_cultural_industries', 'finance_and_insurance', 'real_estate_and_rental_and_leasing', 'professional_scientific_and_technical_services', 'management_of_companies_and_enterprises', 'educational_services', 'health_care_and_social_assistance', 'arts_entertainment_and_recreation', 'accommodation_and_food_services', 'other_services_except_public_administration', 'public_administration'] }
pick_members_dict = {"&pickMembers%5B0%5D=": pick_members_2, "&pickMembers%5B1%5D=": pick_members_3}
filter_names = {"pick_member_1": "Estimate", "pick_member_2": "North American Industry Classification System (NAICS)"}
table_name = "average_weekly_earnings_by_industry"
filter_keys = list(filter_names.keys())
filter_values = list(filter_names.values())

In [4]:
pid="1410022301"
pickMembers1=str(1.3)
pickMembers2=str(2.1)
start_month="01"
start_year="2022"
end_month="12"
end_year="2022"
referencePeriods="20220101%2C20221001"

conn_string = 'mysql+pymysql://' + os.environ["USERNAME"] + ':' + os.environ["PASSWORD"] + '@' + os.environ["HOST"] + '/' + os.environ["DATABASE"] 

keys = list(pick_members_dict.keys())
values = list((pick_members_dict[key]["names"] for key in keys))

pick_members_1_dict = dict(zip(pick_members_dict["&pickMembers%5B0%5D="]["names"], pick_members_dict["&pickMembers%5B0%5D="]["values"]))
pick_members_2_dict = dict(zip(pick_members_dict["&pickMembers%5B1%5D="]["names"], pick_members_dict["&pickMembers%5B1%5D="]["values"]))
result = [(x, y) for x in values[0] for y in values[1]]


In [5]:
print(result)

[('2.2', '3.1'), ('2.2', '3.2'), ('2.2', '3.3'), ('2.2', '3.4'), ('2.2', '3.10'), ('2.2', '3.17'), ('2.2', '3.21'), ('2.2', '3.34'), ('2.2', '3.143'), ('2.2', '3.144'), ('2.2', '3.215'), ('2.2', '3.253'), ('2.2', '3.267'), ('2.2', '3.284'), ('2.2', '3.295'), ('2.2', '3.306'), ('2.2', '3.321'), ('2.2', '3.331'), ('2.2', '3.354'), ('2.2', '3.367'), ('2.2', '3.377'), ('2.2', '3.393')]


In [6]:
import pandas as pd
final_df = pd.DataFrame()

In [7]:
# for x, y in result:
#     estimate = pick_members_1_dict[x]
#     industry = pick_members_2_dict[y]

#     url = 'https://www150.statcan.gc.ca/t1/tbl1/en/tv.action?pid=' + pid + '&pickMembers%5B0%5D='+ x + '&pickMembers%5B1%5D='+ y + '&cubeTimeFrame.startMonth='+ start_month + '&cubeTimeFrame.startYear=' + start_year + '&cubeTimeFrame.endMonth=' + end_month +'&cubeTimeFrame.endYear=' + end_year + '&referencePeriods=' + referencePeriods
#     test_df = scrapper.simple_scrapper(url, filter_names)

    
#     new_test_df = test_df.melt(['date', 'month'], var_name='industry', value_name=industry)
#     if final_df.empty:
#         final_df["month"] = new_test_df["month"]
#         final_df["date"] = new_test_df["date"]
        
#     final_df[industry] = new_test_df[industry]
#     print("Running for: " + estimate + " " + industry)


In [8]:
today = date.today()
four_months_ago = date.today() - relativedelta(months=+4)
start_month=str(four_months_ago.month).zfill(2)
start_year=str(four_months_ago.year)
end_month=str(today.month).zfill(2)
end_year=str(today.year)
referencePeriods = start_year + start_month + "01" + "%2C" + end_year + end_month + "28" 
final_df = pd.DataFrame()

In [13]:

for x, y in result:
    pick_member_1 = pick_members_1_dict[x]
    pick_member_2 = pick_members_2_dict[y]

    url = 'https://www150.statcan.gc.ca/t1/tbl1/en/tv.action?pid=' + pid + '&pickMembers%5B0%5D='+ x + '&pickMembers%5B1%5D='+ y + '&cubeTimeFrame.startMonth='+ start_month + '&cubeTimeFrame.startYear=' + start_year + '&cubeTimeFrame.endMonth=' + end_month +'&cubeTimeFrame.endYear=' + end_year + '&referencePeriods=' + referencePeriods
    test_df = scrapper.simple_scrapper(url, list(filter_names.values())).iloc[-1]
    
    new_test_df = pd.DataFrame(test_df).T.melt(['date', 'month'], var_name='province', value_name=pick_member_2)
    latest_month = new_test_df["month"]
    
    if final_df.empty:
        final_df["month"] = new_test_df["month"]
        final_df["date"] = new_test_df["date"]
        
    final_df[pick_member_2] = new_test_df[pick_member_2]
    print( clean_string(filter_values[0]))
    # print("SELECT FROM " + table_name +  " WHERE '" + clean_string(filter_values[0]) + "' = '" + pick_member_1 + "' AND '" + clean_string(filter_values[1]) + "' = '" + pick_member_2 + "' AND month = '" + latest_month + "';")
    print("Running for: " + pick_member_1 + " " + pick_member_2)

estimate
Running for: average_weekly_earnings industrial_aggregate_including_unclassified_businesses
estimate
Running for: average_weekly_earnings industrial_aggregate_excluding_unclassified_businesses
estimate
Running for: average_weekly_earnings goods_producing_industries
estimate
Running for: average_weekly_earnings forestry_logging_and_support
estimate
Running for: average_weekly_earnings mining_quarrying_and_oil_and_gas_extraction
estimate
Running for: average_weekly_earnings utilities
estimate
Running for: average_weekly_earnings construction
estimate
Running for: average_weekly_earnings manufacturing
estimate
Running for: average_weekly_earnings service_producing_industries
estimate
Running for: average_weekly_earnings trade
estimate
Running for: average_weekly_earnings transportation_and_warehousing
estimate
Running for: average_weekly_earnings information_and_cultural_industries
estimate
Running for: average_weekly_earnings finance_and_insurance
estimate
Running for: average_w

In [ ]:
# average_weekly_earnings = data_pipeline.data_pipeline_job(pid, ["average_weekly_earnings"], pick_members_dict, filter_names, transpose=True)
# average_weekly_earnings.drop(average_weekly_earnings.tail(2).index, inplace = True)